#1 .Configuración del ambiente

In [ ]:
#librerias python y su versión
!python -V
print('------')
!pip show Pandas | grep 'Name\|Version'
print('------')
!pip show Numpy | grep 'Name\|Version'
print('------')
!pip show Matplotlib | grep 'Name\|Version'

##Importacion de bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from urllib.request import urlopen
from bs4 import BeautifulSoup  #de HTML
import requests  #solicitudes HTTP
from IPython.display import clear_output
import time

##Variables Globales

In [ ]:
global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision  #variables globales

#2. Obtención de datos

##Precios históricos de Bitcoin

In [ ]:
#Datos históricos de precios del bitcoin en dólares últimos 7 días (intervaloes de 5 minnutos)

def importar_base_bitcoin():
  global df_bitcoin

  simbolo = "BTC-USD"

  btcDatos = yf.Ticker(simbolo).history(period='7d', interval='5m')
  df_bitcoin = pd.DataFrame(btcDatos)
  return df_bitcoin

In [ ]:
importar_base_bitcoin()
df_bitcoin.head(10)

##Tendencias btc

In [ ]:
#Extraccion del precio actual del BTC de coinmarketCap
def extraer_tendencias():
  global df_bitcoin, df_bitcoin_limpio, precio_actual, tendencia, media_bitcoin,  algoritmo_decision
  headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}
  url = "https://coinmarketcap.com/"
  pageRes = requests.get(url, headers=headers)
  soup = BeautifulSoup(pageRes.content, features="lxml") #para analizar el html
  respuesta= soup.findAll('p',{"class":"sc-4984dd93-0 iqdbQL coin-item-symbol"}) #simbolo bitcoin

  indice = 0
  for item in respuesta:
    if item.getText() == "BTC":
      break

    indice+=1
  lista=[]
  respuesta = soup.findAll("div",{"class":["sc-a0353bbc-0 gDrtaY","sc-a0353bbc-0 gDrtaY fall", "sc-a0353bbc-0 gDrtaY rise" ]}) #si baja o cae el precio
  precio_actual=float(respuesta[indice].getText().replace("$","").replace(",",""))

  respuesta=soup.findAll("span",{"class":["sc-d55c02b-0 iwhBxy","sc-d55c02b-0 iwhBxy"]})
  for item in respuesta[::3]:
    lista.append(item)

  tendencia=""
  if lista[indice].find("span")["class"][0] == "icon-Caret-down":
    tendencia = "baja"
  else:
    tendencia = "alta"


In [ ]:
extraer_tendencias()
print('Bitcoin Precio: ', precio_actual)
print('Tendencia: ', tendencia)

#3. Limpieza de datos

In [ ]:
# Muestra el boxplot de la base sin tratamiento
df_bitcoin.boxplot(['Close'])
plt.title('Distribución del Precio del Bitcoin')

In [ ]:
def limpieza_datos():
    global df_bitcoin, df_bitcoin_limpio, media_bitcoin

    # Crear una copia del dataframe df_bitcoin
    df_bitcoin_limpio = df_bitcoin.copy()

    # Tratar duplicados en el índice (Datetime)
    df_bitcoin_limpio = df_bitcoin_limpio[~df_bitcoin_limpio.index.duplicated(keep='first')]

    # Tratar valores nulos en la columna Close
    df_bitcoin_limpio['Close'].fillna(method='ffill', inplace=True)

    # Eliminar registros con Volume de transacción igual o menor a 0
    df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio['Volume'] > 0]

    # Identificar y eliminar outliers en el precio del Bitcoin (Close)
    q1 = df_bitcoin_limpio['Close'].quantile(0.25)
    q3 = df_bitcoin_limpio['Close'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df_bitcoin_limpio = df_bitcoin_limpio[(df_bitcoin_limpio['Close'] >= lower_bound) & (df_bitcoin_limpio['Close'] <= upper_bound)]

    # Calcular el precio promedio (Close)
    media_bitcoin = df_bitcoin_limpio['Close'].mean()

In [ ]:
def grafico_boxplot(dataframe):
    # Obtener los valores máximo, mediana, mínimo, Q1 y Q3
    max_val = dataframe['Close'].max()
    median_val = dataframe['Close'].median()
    min_val = dataframe['Close'].min()
    q1_val = dataframe['Close'].quantile(0.25)
    q3_val = dataframe['Close'].quantile(0.75)

    # Crear gráfico boxplot horizontal
    plt.figure(figsize=(10, 6))
    plt.boxplot(dataframe['Close'], vert=True, patch_artist=True)

    # Personalizar el gráfico
    plt.title('Distribución del Precio del Bitcoin')
    plt.xlabel('Cierre')
    plt.ylabel('Precio de Bitcoin')

    # Agregar guías para valor máximo, mediana, mínimo, Q1 y Q3
    plt.axhline(y=max_val, color='red', linestyle='--')
    plt.axhline(y=median_val, color='white', linestyle='--')
    plt.axhline(y=min_val, color='green', linestyle='--')
    plt.axhline(y=q1_val, color='orange', linestyle='--', label=f'Q1: {q1_val:.2f}')
    plt.axhline(y=q3_val, color='purple', linestyle='--', label=f'Q3: {q3_val:.2f}')

    # Etiquetas de los valores en las líneas
    plt.annotate(f'Máximo: {max_val:.2f}', xy=(1, max_val), xytext=(10, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points', color='red')
    plt.annotate(f'Mediana: {median_val:.2f}', xy=(1, median_val), xytext=(10, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points', color='white')
    plt.annotate(f'Mínimo: {min_val:.2f}', xy=(1, min_val), xytext=(10, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points', color='green')
    plt.annotate(f'Q1: {q1_val:.2f}', xy=(1, q1_val), xytext=(10, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points', color='orange')
    plt.annotate(f'Q3: {q3_val:.2f}', xy=(1, q3_val), xytext=(10, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points', color='purple')

    # Cambiar colores del boxplot
    box_color = 'white'
    whisker_color = 'gray'
    flier_color = 'red'

    for patch in plt.boxplot(dataframe['Close'], vert=True, patch_artist=True)['boxes']:
        patch.set(facecolor=box_color, linewidth=1.5)

    for whisker in plt.boxplot(dataframe['Close'], vert=True, patch_artist=True)['whiskers']:
        whisker.set(color=whisker_color, linewidth=1.5)

    for cap in plt.boxplot(dataframe['Close'], vert=True, patch_artist=True)['caps']:
        cap.set(color=whisker_color, linewidth=1.5)

    for flier in plt.boxplot(dataframe['Close'], vert=True, patch_artist=True)['fliers']:
        flier.set(marker='o', color=flier_color, alpha=0.8)

    # Mostrar el gráfico
    plt.show()

grafico_boxplot(df_bitcoin_limpio)

In [ ]:
limpieza_datos()

#4. Tomar decisiones

In [ ]:
def tomar_decisiones():
  global algoritmo_decision

  if precio_actual >= media_bitcoin and tendencia == 'baja':
      algoritmo_decision = 'Vender'
  elif precio_actual < media_bitcoin and tendencia == 'alta':
      algoritmo_decision = 'Comprar'
  else:
      algoritmo_decision = 'esperar'



In [ ]:
tomar_decisiones()

In [ ]:
print("Decisión del algoritmo:", algoritmo_decision)

#5. Visualización

In [ ]:
def visualizacion():
  global df_bitcoin


  #Agregando columna promedio
  df_bitcoin['Promedio'] = media_bitcoin


  #Dibujando el grafico
  plt.style.use('ggplot')
  plt.rcParams['axes.grid'] = True
  plt.figure(figsize=(16, 5))
  plt.title("Precio BTC")
  plt.plot(df_bitcoin.index, df_bitcoin['Close'], label='Precio del Bitcoin', color='blue')
  plt.plot(df_bitcoin.index, df_bitcoin['Promedio'], label='Precio Promedio', linestyle='--', color='orange')

  plt.text(df_bitcoin.index[-1], media_bitcoin + 30, f'Promedio: {media_bitcoin:.2f}', color='green', verticalalignment='bottom', horizontalalignment='right')

  #tomador de decisiones en grafico
  if algoritmo_decision == 'Vender':
    plt.annotate('Vender', xy=(df_bitcoin.index[-1], df_bitcoin['Close'].iloc[-1]), xytext=(-20, 30),
                  textcoords='offset points', arrowprops=dict(arrowstyle="->", color='red'))
  elif algoritmo_decision == 'Comprar':
    plt.annotate('Comprar', xy=(df_bitcoin.index[-1], df_bitcoin['Close'].iloc[-1]), xytext=(-20, -30),
                  textcoords='offset points', arrowprops=dict(arrowstyle="->", color='green'))


  plt.xlabel('Fecha')
  plt.ylabel('Precio')
  plt.legend()

  plt.show()


In [ ]:
visualizacion()

#6. Automatización

In [ ]:
def automatizacion():
  while (True):
    clear_output()
    importar_base_bitcoin()
    extraer_tendencias()
    limpieza_datos()
    tomar_decisiones()
    visualizacion()
    time.sleep(300)

automatizacion()